Una empresa desea evaluar si hay diferencias en el desempeño de empleados provenientes de tres diferentes programas de capacitación: Grupo X, Grupo Y y Grupo Z. Se evalúa el desempeño con una calificación final (sobre 100 puntos) después de un mes de trabajo.

Los datos recolectados para cada grupo son los siguientes:
* Grupo X: 12 empleados
* Grupo Y: 15 empleados
* Grupo Z: 10 empleados

Cada grupo tiene un conjunto de calificaciones individuales.

**Objetivo:** Determinar si existen diferencias significativas en el promedio de desempeño entre los tres programas de capacitación utilizando un análisis ANOVA de un factor.

1. Hipótesis:
$$H_0: \mu_X=\mu_Y=\mu_Z$$
$$H_a: \mu_i\neq\mu_j,\hspace{0.5cm}\forall i\neq j$$

In [2]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [3]:
# 2.Estadísticas descriptivas:
# a) Calcula la media de cada grupo.

# Cargar el archivo Excel
df = pd.read_excel("datos_anova_desempeno.csv.xlsx")

group_means = df.groupby("grupo")["valor"].mean()
global_mean = df["valor"].mean()

print("Medias por grupo:\n", group_means)

Medias por grupo:
 grupo
X    76.479777
Y    77.106921
Z    77.736647
Name: valor, dtype: float64


In [4]:
# b) Calcula la media global.

print("\nMedia global:", global_mean)


Media global: 77.07371910813394


In [5]:
# 3.ANOVA:
# a) Calcula la Suma de Cuadrados Total (SCT).

SCT = ((df["valor"] - global_mean) ** 2).sum()
print("\nSCT:", SCT)


SCT: 647.9979146302204


In [6]:
# b) Calcula la Suma de Cuadrados Entre Grupos (SCE).

SCE = sum(
    len(df[df["grupo"] == g]) * (mean - global_mean) ** 2
    for g, mean in group_means.items()
)
print("SCE:", SCE)

SCE: 8.644482149304004


In [7]:
# c) Calcula la Suma de Cuadrados Dentro de los Grupos (SCD).

SCD = sum(
    ((df[df["grupo"] == g]["valor"] - mean) ** 2).sum()
    for g, mean in group_means.items()
)
print("SCD:", SCD)

SCD: 639.3534324809162


In [8]:
# d) Verifica que SCT = SCE + SCD.

print("Verificación SCT ≈ SCE + SCD:", SCT - (SCE + SCD))

Verificación SCT ≈ SCE + SCD: 1.1368683772161603e-13


In [9]:
# 4.Estadístico F:
# a) Calcula los cuadrados medios: MCE y MCD.

# Grados de libertad
k = df["grupo"].nunique()
n = len(df)

df_entre = k - 1
df_dentro = n - k

# Cuadrados medios
MCE = SCE / df_entre
MCD = SCD / df_dentro
print("\nMCE:", MCE)
print("MCD:", MCD)


MCE: 4.322241074652002
MCD: 18.80451272002695


In [11]:
# b) Calcula el estadístico F.

F = MCE / MCD
print("Estadístico F:", F)

Estadístico F: 0.22985126703383185


In [12]:
# 5.Decisión estadística:
# a) Encuentra el valor crítico o el p-value para el estadístico F.

p_value = 1 - stats.f.cdf(F, df_entre, df_dentro)
print("p-value:", p_value)

p-value: 0.7958764951439442


In [13]:
# b) Compara con el nivel de significancia (α = 0.05). ¿Se rechaza H₀?

alpha = 0.05
if p_value < alpha:
    print("\nSe rechaza H₀: Hay diferencias significativas entre los grupos.")
else:
    print("\nNo se rechaza H₀: No hay diferencias significativas entre los grupos.")


No se rechaza H₀: No hay diferencias significativas entre los grupos.


6. Interpretación:

a) ¿Qué concluyes sobre los programas de capacitación?


b) ¿Cuál(es) parecen ser más efectivos?

In [15]:
# 7.Análisis post-hoc (Tukey HSD):
# a) Realiza la prueba de Tukey para comparar pares de grupos.

# ANOVA usando statsmodels (para Tukey)
modelo = ols('valor ~ grupo', data=df).fit()
anova_table = sm.stats.anova_lm(modelo, typ=2)
print("\nTabla ANOVA con statsmodels:\n", anova_table)

# Tukey HSD (post-hoc)
tukey = pairwise_tukeyhsd(endog=df['valor'], groups=df['grupo'], alpha=0.05)
print("\nResultado de la prueba de Tukey HSD:\n")
print(tukey)


Tabla ANOVA con statsmodels:
               sum_sq    df         F    PR(>F)
grupo       8.644482   2.0  0.229851  0.795876
Residual  639.353432  34.0       NaN       NaN

Resultado de la prueba de Tukey HSD:

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     X      Y   0.6271 0.9262 -3.4883 4.7426  False
     X      Z   1.2569 0.7784  -3.293 5.8067  False
     Y      Z   0.6297 0.9328 -3.7084 4.9678  False
---------------------------------------------------


b) Indica qué diferencias entre pares son significativas.

Según los resultados, ¿qué programa recomendarías a la empresa como el más efectivo en promedio para formar empleados?